In [22]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [23]:
%load_ext dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [24]:
import os

In [25]:
!pwd
!ls -l ./

/mnt/datadrive01/prj/AzureChestXRayNoAML/code
total 3232
-rwxrwxrwx 1 loginVM_001 loginVM_001  401244 Sep 24 01:38 00_create_docker_image.html
-rwxrwxrwx 1 loginVM_001 loginVM_001 2621340 Sep 24 05:20 00_create_docker_image.ipynb
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 24 05:12 01_DataPrep
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 24 05:18 02_Model
drwxrwxrwx 2 loginVM_001 loginVM_001    4096 Sep 24 04:24 03_Deployment
-rwxrwxrwx 1        1000 users        255403 Sep 23 23:56 edit_python_files.html
-rwxrwxrwx 1        1000 users          6602 Sep 23 23:58 edit_python_files.ipynb
drwxrwxrwx 3        1000 users          4096 Sep 24 02:07 notShared
drwxrwxrwx 3 loginVM_001 loginVM_001    4096 Sep 23 23:10 src


In [26]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu"

Overwriting .env


In [33]:
%%writefile ./../docker/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER keras
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy==1.14.5 && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile


In [34]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':1.0.0'
docker_file_location = os.path.join(*(['.', '..','docker', 'dockerfile']))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu:1.0.0'

'./../docker/dockerfile'

In [35]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu:1.0.0 -f ./../docker/dockerfile .
Sending build context to Docker daemon  11.06MB
Step 1/16 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> 288866c2c06c
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> bc9e7c9ac9dd
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 3c415595ae4e
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> e08bdb87f797
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 3e2afc791489
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" |

  Running setup.py bdist_wheel for networkx: started
  Running setup.py bdist_wheel for networkx: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/3c/33/ae/db8cd618e62f87594c13a5483f96e618044f9b01596efd013f
  Running setup.py bdist_wheel for html5lib: started

  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Running setup.py bdist_wheel for toolz: started
  Running setup.py bdist_wheel for toolz: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx termcolor gast absl-py html5lib toolz
Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    appdirs:           1.4.3-py36h28b3542_0  
    attrs:             18.2.0-py36h28b3542_0 
    automat:           0.7.0-py36_0          
    backcall:          0.1.0-py36_0          
    bcolz:             1.2.1-py36h04863e7_0  
    blas:              1.0-mkl               
    bleach:            2.1.4-py36_0          
    blosc:     

py-opencv-3.4. 100% |###############################| Time: 0:00:00  44.35 MB/s
pygpu-0.7.6-py 100% |###############################| Time: 0:00:00  55.66 MB/s
scipy-1.1.0-py 100% |###############################| Time: 0:00:00  58.30 MB/s
dask-0.19.1-py 100% |###############################| Time: 0:00:00   8.53 MB/s
opencv-3.4.2-p 100% |###############################| Time: 0:00:00  25.99 MB/s
pytables-3.4.4 100% |###############################| Time: 0:00:00  29.36 MB/s
scikit-learn-0 100% |###############################| Time: 0:00:00  62.18 MB/s
bcolz-1.2.1-py 100% |###############################| Time: 0:00:00  57.83 MB/s
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - imageio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge

  Running setup.py bdist_wheel for pytest-cache: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/e4/9e/28/59d0a23800e49808f17920c5922252c661966ca4a36db5ee38
Successfully built pytest-pep8 pytest-cache
  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-x0reglpr
  Running setup.py bdist_wheel for Keras: started
  Running setup.py bdist_wheel for Keras: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-wkp7x1uo/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras
  Cloning https://github.com/aleju/imgaug to /tmp/pip-req-build-w1pa4xs0
  Running setup.py bdist_wheel for imgaug: started
  Running setup.py bdist_wheel for imgaug: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-6f4fyoq4/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com

html5lib-1.0.1-py36_0.tar.bz2                184 KB
qt-5.9.6-h52aff34_0.tar.bz2                 86.7 MB
pyqt-5.9.2-py36h05f1152_2.tar.bz2            5.6 MB
ca-certificates-2018.8.24-ha4d7672_0.tar.bz2     136 KB
pixman-0.34.0-hceecf20_3.tar.bz2             598 KB
harfbuzz-1.8.8-hffaf4a1_0.tar.bz2            863 KB
pexpect-4.6.0-py36_0.tar.bz2                  77 KB
tk-8.6.8-hbc83047_0.tar.bz2                  3.1 MB
libgfortran-ng-7.3.0-hdf63c60_0.tar.bz2      1.3 MB
bleach-2.1.4-py36_0.tar.bz2                   33 KB
pyzmq-17.1.2-py36h14c3975_0.tar.bz2          454 KB
jsonschema-2.6.0-py36_0.tar.bz2               62 KB
automat-0.7.0-py36_0.tar.bz2                  52 KB
cffi-1.10.0-py36_0.tar.bz2                   341 KB
notebook-5.6.0-py36_0.tar.bz2                7.4 MB
bzip2-1.0.6-h14c3975_5.tar.bz2               414 KB
ipykernel-4.9.0-py36_1.tar.bz2               146 KB
scikit-learn-0.19.2-py36h4989274_0.tar.bz2     5.2 MB
pyasn1-modules-0.2.2-py36_0.tar.bz2           86 KB
cytool

Removing intermediate container 30e69b926fb0
 ---> 69238b7802b0
Step 14/16 : ENV PYTHONPATH='/src/:$PYTHONPATH'
 ---> Running in c0d1522658d9
Removing intermediate container c0d1522658d9
 ---> f04c05574587
Step 15/16 : WORKDIR /src
Removing intermediate container 3fa5af62d3b8
 ---> c0ae1c1b72b6
Step 16/16 : EXPOSE 8888
 ---> Running in fff81d22faad
Removing intermediate container fff81d22faad
 ---> 24dd48cfb793
Successfully built 24dd48cfb793
Successfully tagged georgedockeraccount/chestxray-no-aml-gpu:1.0.0


In [30]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu]

c84a27c6: Preparing 
a89b6996: Preparing 
a1f3e5d0: Preparing 
9618113d: Preparing 
e517b5f3: Preparing 
e5d9d370: Preparing 
6d4f5f2e: Preparing 
3c790502: Preparing 
ac05da10: Preparing 
19f38a85: Preparing 
d6639b96: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 
84a27c6: Pushing  244.4MB/5.589GB15A^C


In [31]:
!echo sudo nvidia-docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

sudo nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [32]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 320173 bytes to 00_create_docker_image.html
